In [1]:
import torch
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from PIL import Image

In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [3]:
# Parameters
num_classes = 2  # Update this based on the number of classes in your dataset
batch_size = 32
number_of_epochs = 30
learning_rate = 0.001

In [ ]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:
# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)


In [6]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# Initialize the model
model = models.mobilenet_v2(pretrained=True)


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Modify the classifier to fit the number of classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

In [9]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [11]:
# Train the model
for epoch in range(number_of_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{number_of_epochs}], Loss: {epoch_loss:.4f}')

    # Validate the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        print(f'Accuracy of the model on the validation images: {100 * correct / total:.2f}%')

Epoch [1/30], Loss: 0.7004
Accuracy of the model on the validation images: 64.29%
Epoch [2/30], Loss: 0.5288
Accuracy of the model on the validation images: 57.14%
Epoch [3/30], Loss: 0.3045
Accuracy of the model on the validation images: 60.71%
Epoch [4/30], Loss: 0.1870
Accuracy of the model on the validation images: 64.29%
Epoch [5/30], Loss: 0.1063
Accuracy of the model on the validation images: 60.71%
Epoch [6/30], Loss: 0.1889
Accuracy of the model on the validation images: 57.14%
Epoch [7/30], Loss: 0.2341
Accuracy of the model on the validation images: 67.86%
Epoch [8/30], Loss: 0.2378
Accuracy of the model on the validation images: 75.00%
Epoch [9/30], Loss: 0.2839
Accuracy of the model on the validation images: 67.86%
Epoch [10/30], Loss: 0.4946
Accuracy of the model on the validation images: 85.71%
Epoch [11/30], Loss: 0.3623
Accuracy of the model on the validation images: 71.43%
Epoch [12/30], Loss: 0.3030
Accuracy of the model on the validation images: 71.43%
Epoch [13/30]

In [12]:
# Save the model, optional
torch.save(model.state_dict(), f'Pth_Files/Mobilenet_model_{number_of_epochs}.keras')

In [13]:
def predict_class(image_path):
    image = Image.open(image_path).convert('RGB')  # Convert image to RGB if not

    # Apply the transformations
    input_image = transform(image).unsqueeze(0)  # Add batch dimens

    # Predict the class
    with torch.no_grad():  # No need to track gradients for inference
        output = model(input_image)
        _, predicted = torch.max(output.data, 1)  # Get the index of the max log-probability

    # Assuming you have a list of class names corresponding to your label indices
    class_names = ['class1', 'class2']  # Fill in your actual class names
    predicted_class = class_names[predicted.item()]

    print(f'Predicted Class: {predicted_class}')


In [14]:
image_1 = test_dir + "/Agaricus_abruptibulbus/Agaricus_abruptibulbus_1.jpg"
image_2 = test_dir + "/Amanita_excelsa/Amanita_excelsa_1.jpg"


In [15]:
predict_class(image_1)
predict_class(image_2)

Predicted Class: class2
Predicted Class: class2
